In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D,Flatten,Dense,Activation,Add,BatchNormalization,MaxPooling2D,AveragePooling2D,Input,ZeroPadding2D
from keras.initializers import glorot_uniform
from keras.models import Model
import keras
from keras.regularizers import l2
(X_train,y_train),(X_test,y_test)=keras.datasets.cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def identity_block(X,f,filters):
    
    X_temp = X
    F1,F2 = filters
    
    X = Conv2D(F1,(f,f), strides=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [0]:
def conv_block(X,f,filters,s):
    X_temp = X
    F1,F2 = filters
    
    X_temp=Conv2D(F2,(1,1), strides=(s,s), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X_temp)
    X_temp=BatchNormalization(axis=3)(X_temp)
    
    X = Conv2D(F1,(f,f), strides=(s,s),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X = Conv2D(F2,(f,f), strides=(1,1), padding="same", kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([X,X_temp])
    X=Activation('relu')(X)
    
    return X
    

In [0]:
def Res44(input_shape,classes):
    
    X_input=Input(input_shape)
    X=ZeroPadding2D((1,1))(X_input)
    X=Conv2D(16,(3,3), strides=(1,1), kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    #X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    X=identity_block(X,3,(16,16))
    
    X=conv_block(X,3,(32,32),2)
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))
    X=identity_block(X,3,(32,32))

    X=conv_block(X,3,(64,64),2)
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    X=identity_block(X,3,(64,64))
    
    #X=conv_block(X,3,(256,256),2)
    #X=identity_block(X,3,(256,256))
    
    
    #X=conv_block(X,3,(512,512),2)    #commented to reduce params
    #X=identity_block(X,3,(512,512))  # commented out to reduce params
    
    
    X = AveragePooling2D((8, 8))(X)
    
    X = Flatten()(X)
    
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(X)
    
    model=Model(inputs = X_input, outputs = X, name='ResNet34')
    
    return model

In [0]:
model = Res44((32,32,3),10)

In [6]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 34, 34, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
___________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [0]:
image_gen= keras.preprocessing.image.ImageDataGenerator( 
    horizontal_flip=True,
    zca_epsilon=1e-06,
    fill_mode="nearest",
    width_shift_range=0.1,
    height_shift_range=0.1)

image_gen.fit(X_train)

In [12]:
model.fit_generator(image_gen.flow(X_train, y_train, batch_size=128),
                        validation_data=(X_test, y_test),
                        epochs=30,workers=4)

Epoch 1/30
391/391 [==============================] - 129s 331ms/step - loss: 0.4017 - accuracy: 0.9311 - val_loss: 0.8902 - val_accuracy: 0.8079
Epoch 2/30
391/391 [==============================] - 128s 329ms/step - loss: 0.4034 - accuracy: 0.9294 - val_loss: 0.7936 - val_accuracy: 0.8329
Epoch 3/30
391/391 [==============================] - 128s 327ms/step - loss: 0.4020 - accuracy: 0.9298 - val_loss: 0.7528 - val_accuracy: 0.8397
Epoch 4/30
391/391 [==============================] - 129s 330ms/step - loss: 0.4014 - accuracy: 0.9309 - val_loss: 0.6602 - val_accuracy: 0.8507
Epoch 5/30
391/391 [==============================] - 129s 329ms/step - loss: 0.3978 - accuracy: 0.9312 - val_loss: 0.7744 - val_accuracy: 0.8411
Epoch 6/30
391/391 [==============================] - 129s 329ms/step - loss: 0.3996 - accuracy: 0.9323 - val_loss: 0.7559 - val_accuracy: 0.8370
Epoch 7/30
391/391 [==============================] - 128s 326ms/step - loss: 0.3906 - accuracy: 0.9342 - val_loss: 0.6646 -